In [0]:
import pandas as pd
import numpy as np

rotations = pd.read_csv("rotations.csv")
rotations = pd.DataFrame(rotations)

spots = pd.read_csv("spots.csv")
spots = pd.DataFrame(spots)

In [122]:
rotations.head()

,Start,End,Name
0,6:00 AM,12:00 PM,Morning
1,12:00 PM,4:00 PM,Afternoon
2,3:00 PM,8:00 PM,Prime


In [123]:
spots.head()

,Date,Time,Creative,Spend,Views
0,01/02/2016,8:30 AM,TEST001H,120.5,100
1,01/02/2016,11:30 AM,TEST001H,240.5,110
2,01/02/2016,3:30 PM,TEST002H,500.0,80
3,01/02/2016,3:34 PM,TEST002H,400.0,90
4,01/02/2016,3:40 PM,TEST001H,400.0,110


In [124]:
# Convert am/pm to 24-hour format
rotations["Start"] = pd.to_datetime(rotations.Start).dt.strftime("%H:%M:%S").str.split(":")
rotations["End"] = pd.to_datetime(rotations.End).dt.strftime("%H:%M:%S").str.split(":")

# Convert hours to minutes
rotations["Start_in_Minutes"] = rotations.Start.apply(lambda x: int(x[0]) * 60 + int(x[1]))
rotations["End_in_Minutes"] = rotations.End.apply(lambda x: int(x[0]) * 60 + int(x[1]))
rotations = rotations.drop(columns=["Start", "End"])

rotations.head()


,Name,Start_in_Minutes,End_in_Minutes
0,Morning,360,720
1,Afternoon,720,960
2,Prime,900,1200


In [125]:
##### Assumption #1 #####
# Among a given set of rotation time ranges, it is clear that "Afternoon" and "Prime" rotations overlap. 
# To circumvent this, "Prime" rotation has been prioritized and "Afternoon" rotation's "End_in_Minutes" has been updated with "Prime" rotation's "Start_in_Minutes"
rotations["End_in_Minutes"] = np.where(rotations.End_in_Minutes > rotations.Start_in_Minutes.shift(-1),rotations.Start_in_Minutes.shift(-1),rotations.End_in_Minutes).astype("int64")

rotations.head()


,Name,Start_in_Minutes,End_in_Minutes
0,Morning,360,720
1,Afternoon,720,900
2,Prime,900,1200


In [126]:
# Convert am/pm to 24-hour format
spots["Time"] = pd.to_datetime(spots.Time).dt.strftime("%H:%M:%S").str.split(":")
# Date reformatting
spots["Date"] = pd.to_datetime(spots.Date)
# Convert hours to minutes
spots["Time_in_Minutes"] = spots.Time.apply(lambda x: int(x[0]) * 60 + int(x[1]))
# Create "Weekday" column 
spots["Weekday"] = spots.Date.dt.weekday_name
spots = spots.drop(columns=["Time", "Date"])
# Calculate CPV
spots["CPV"] = spots.apply(lambda row: row.Spend / row.Views,axis=1)

spots.head()

,Creative,Spend,Views,Time_in_Minutes,Weekday,CPV
0,TEST001H,120.5,100,510,Saturday,1.205000
1,TEST001H,240.5,110,690,Saturday,2.186364
2,TEST002H,500.0,80,930,Saturday,6.250000
3,TEST002H,400.0,90,934,Saturday,4.444444
4,TEST001H,400.0,110,940,Saturday,3.636364


In [135]:
# A method to determine what rotation does given time_in_minutes represent
def find_rotation_interval(time_in_minutes):
  return next(rotation for rotation in np.where(np.logical_and(rotations.Start_in_Minutes <= time_in_minutes,\
                                                               rotations.End_in_Minutes >= time_in_minutes),rotations.Name, "") if rotation)
    
# Create a new column "Rotation" and populate with corresponding rotation names
spots["Rotation"] = spots.apply(lambda row: find_rotation_interval(row.Time_in_Minutes), axis=1)

cpv_by_creative = spots.CPV.groupby(spots.Creative).mean()
print("#### CPV by creative ####")
print("")
print(cpv_by_creative)
print("")
print("")
print("#### CPV by rotation by day ####")
print("")
cpv_by_rotation_by_day = spots.groupby(["Weekday", "Rotation"]).CPV.mean()
print(cpv_by_rotation_by_day)



#### CPV by creative ####

Creative
TEST001H    2.631932
TEST002H    4.342593
Name: CPV, dtype: float64


#### CPV by rotation by day ####

Weekday   Rotation
Saturday  Morning     1.695682
          Prime       4.776936
Tuesday   Morning     3.500000
          Prime       2.333333
Name: CPV, dtype: float64
